In [8]:
import numpy as np
import pandas as pd
from datetime import timedelta
from datetime import date
from datetime import datetime
import time
from datascience import *
from scipy.spatial import distance

from IPython.display import display
from IPython.display import HTML

import ipywidgets as widgets
from ipywidgets import IntSlider

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [6]:
# Constansts

output_directory = "output/"

COL_PURCHASE_DATE = 'PURCHASE DATE'
COL_SOLD_DATE = 'SOLD DATE'
COL_PURCHASE_PRICE = 'PURCHASE PRICE'
COL_SOLD_PRICE = 'SOLD PRICE'

COL_PRICE_PERCENT = 'PRICE CHANGE %'

# Columns for standard units
COL_PURCHASE_DATE_SU = 'PURCHASE DATE SU'
COL_PURCHASE_PRICE_SU = 'PURCHASE PRICE SU'
COL_SOLD_DATE_SU = 'SOLD DATE SU'


# Load Model

In [3]:
df = pd.read_csv(output_directory+'condo_sales.csv')

# DATE is  but should be datetime
df[ COL_PURCHASE_DATE] = pd.to_datetime(df[COL_PURCHASE_DATE], errors='coerce')
df[ COL_SOLD_DATE] = pd.to_datetime(df[COL_SOLD_DATE], errors='coerce')

condo_sales = Table.from_df(df)

# Price Predictor

Enter the **purchase price and date** of an apartment to see it's current price and comparable sales

In [10]:
# Columns used to the calculate the distance between two point i.e. 2 properties that were purchased and sold. 
# We picked the purchase date & price and the sold date converted to standard units as the important columns
# to use for calculating the distance.
distance_columns = [COL_PURCHASE_DATE_SU, COL_PURCHASE_PRICE_SU, COL_SOLD_DATE_SU]


purchase_dates_timestamps = [ date.timestamp() for date in condo_sales.column(COL_PURCHASE_DATE)]
sold_dates_timestamps = [ date.timestamp() for date in condo_sales.column(COL_SOLD_DATE)]



def all_distances(training, new_point):
    """Returns an array of distances
    between each point in the training set
    and the new point (which is a row of attributes)"""
    attributes = training.select(distance_columns)
    return distance.cdist( attributes.to_array().tolist(), [new_point]).flatten()

def table_with_distances(training, new_point):
    """Augments the training table 
    with a column of distances from new_point"""
    return training.with_column('Distance', all_distances(training, new_point))

def closest(training, new_point, k):
    """Returns a table of the k rows of the augmented table
    corresponding to the k smallest distances"""
    with_dists = table_with_distances(training, new_point)
    sorted_by_distance = with_dists.sort('Distance')
    topk = sorted_by_distance.take(np.arange(k))
    return topk

def estimate(training, purchase_point, k):
    """Estimate a price based on nearest neighbours"""
    close_points = closest(condo_sales, purchase_point, k)
    avg_price_change = np.mean(close_points.column(COL_PRICE_PERCENT))
    return avg_price_change


def predict_sale_price(purchase_price, purchase_date: datetime.date):
    """Predict the current value based on the purchase price and purchase date."""
    
    purchase_date_timestamp = time.mktime(purchase_date.timetuple())

    # Set sold date to now
    sold_date = datetime.now().timestamp()

    # calcuate all values in standard units.
    purchase_date_su = (purchase_date_timestamp - np.mean(purchase_dates_timestamps)) / np.std(purchase_dates_timestamps)
    purchase_price_su = (purchase_price - np.mean(condo_sales.column(COL_PURCHASE_PRICE))) / np.std(condo_sales.column(COL_PURCHASE_PRICE))
    sold_date_su = (sold_date - np.mean(sold_dates_timestamps)) / np.std(sold_dates_timestamps)

    # debugging
    # print(purchase_date_su, purchase_price_su, sold_date_su)

    # create a target row to get estimates
    target_row = Table(distance_columns)
    target_row = target_row.with_row([purchase_date_su,purchase_price_su,sold_date_su ])
    
    price_change_percent = estimate(condo_sales, target_row.row(0), 10)
    
    price = purchase_price * (1 + price_change_percent / 100)
    
    display(HTML("<H1>Current Price: ${:,.0f} </H1>".format(price)))
#     display(HTML("<H2>Estimated Price Change: {:.2f}% </H2>".format(price_change_percent)))
    
    display(HTML("<H1>Compareable sales</H1>"))
    closest(condo_sales, target_row.row(0), 10).drop(distance_columns).show()
    
style = {'description_width': 'initial'}

purchase_price_slider = widgets.IntSlider(
    description="Purchase Price",
    value=750000,
    min=100000,
    max=6000000,
    step=10000,
    disabled=False,
    continuous_update=False,
    style=style
    
)

purchase_date_picker = widgets.DatePicker(
    description='Purchase Date',
    disabled=False,
    style=style,
    value=date(2010, 1, 1)
)

In [11]:
style = {'description_width': 'initial'}
IntSlider(description='A too long description', style=style)

_ = interact( predict_sale_price, purchase_price=purchase_price_slider, purchase_date=purchase_date_picker, )

interactive(children=(IntSlider(value=750000, continuous_update=False, description='Purchase Price', max=60000…